In [2]:
import boto3
import csv
import pandas as pd

Query Athena catalog

In [ ]:
client = boto3.client('athena', region_name="us-west-2")
resp = client.start_query_execution(
        QueryString="show databases",
        ResultConfiguration={
            'OutputLocation': 's3://omop5/output/'
        }
    )
print(resp)

Read a file from S3 using boto3

In [ ]:
 
s3client = boto3.client('s3', region_name='us-west-2')
 
# These define the bucket and object to read
bucketname = "omop5" 
file_to_read = "sample1/csv/Beneficiary_Summary/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv" 

#Create a file object using the bucket and object key. 
fileobj = s3client.get_object(
    Bucket=bucketname,
    Key=file_to_read
    ) 
# open the file object and read it into the variable filedata. 
filedata = fileobj['Body'].read()

# file data will be a binary stream.  We have to decode it 
contents = filedata.decode('utf-8')

# Once decoded, you can treat the file as plain text if appropriate 
print(contents)

In [ ]:
my_csv = csv.reader(contents.split("\r\n"))

In [ ]:
cols = next(my_csv)
print(cols)

In [ ]:
rows = [row for row in my_csv]

df = pd.DataFrame(data=rows, columns=cols)
df.head()

In [ ]:
df.shape

Directly read a csv file from S3

In [3]:

df1 = pd.read_csv('s3://omop5/sample1/csv/Beneficiary_Summary/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv')
df1.shape

(116352, 32)

In [4]:
df1.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
0,00013D2EFD8E45D1,19230501,NaN,1,1,0,26,950,12,12,...,2,0.0,0.0,0.0,50.0,10.0,0.0,0.0,0.0,0.0
1,00016F745862898F,19430101,NaN,1,1,0,39,230,12,12,...,2,0.0,0.0,0.0,0.0,0.0,0.0,700.0,240.0,0.0
2,0001FDD721E223DC,19360901,NaN,2,1,0,39,280,12,12,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00021CA6FF03E670,19410601,NaN,1,5,0,6,290,0,0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00024B3D2352D2D0,19360801,NaN,1,1,0,52,590,12,12,...,2,0.0,0.0,0.0,30.0,40.0,0.0,220.0,80.0,0.0


Directly a dataframe to S3 as a csv file

In [ ]:
df1.to_csv('s3://omop5/test1.csv', index=False)

In [39]:
from datetime import date
import awswrangler as wr

s3_bucket_name = "omop5"
s3_path = f"s3://{s3_bucket_name}/test.parquet"

df = pd.DataFrame({
    "id": [1, 2],
    "value": ["foo", "boo"],
    "date": [date(2020, 1, 1), date(2020, 1, 2)]
})

wr.pandas.to_parquet(
    dataframe=df,
    path=s3_path,
    mode="overwrite",
    preserve_index=False
)

wr.pandas.read_parquet(path=s3_path)

,id,value,date
0,1,foo,2020-01-01
1,2,boo,2020-01-02
